# Product Review and Recommendation System
## Final Project Part 2




In [1]:
import sqlite3         
import pandas as pd

In [2]:
conn = sqlite3.connect('amazon_project.db')   
c = conn.cursor()

In [3]:
amazon_data = pd.read_csv(r'amazon.csv')       
amazon_data.to_sql('amazon_data', conn, if_exists='append', index = False) 

1465

## Cleaning Data

In [5]:

c.execute('''           
UPDATE amazon_data
SET discounted_price = CAST(REPLACE(REPLACE(discounted_price, "₹", ""), ",", "") AS float),
    actual_price = CAST(REPLACE(REPLACE(actual_price, "₹", ""), ",", "") AS float);
''')



In [11]:

c.execute('''
UPDATE amazon_data
SET discount_percentage = CAST(REPLACE(discount_percentage, '%', '') AS float) / 100;
''')



In [14]:

c.execute('''
SELECT rating, COUNT(*) as count
FROM amazon_data
GROUP BY rating;
''')
rating_df = pd.DataFrame(c.fetchall(),columns=['Rating',
'COUNT'])
print(rating_df)

   Rating  COUNT
0       2      1
1     2.3      1
2     2.6      1
3     2.8      2
4     2.9      1
5       3      1
6     3.0      3
7     3.1      4
8     3.2      2
9     3.3     16
10    3.4     10
11    3.5     26
12    3.6     35
13    3.7     42
14    3.8     86
15    3.9    123
16      4     52
17    4.0    129
18    4.1    244
19    4.2    228
20    4.3    230
21    4.4    123
22    4.5     75
23    4.6     17
24    4.7      6
25    4.8      3
26    5.0      3
27      |      1


In [15]:

c.execute('''
UPDATE amazon_data
SET rating = CAST(REPLACE(rating, '|', '4.0') AS float);
''')

In [20]:

c.execute('''
SELECT rating, COUNT(*) as count
FROM amazon_data
GROUP BY rating;
''')
rating_df = pd.DataFrame(c.fetchall(),columns=['Rating',
'COUNT'])
print(rating_df)

   Rating  COUNT
0     2.0      1
1     2.3      1
2     2.6      1
3     2.8      2
4     2.9      1
5     3.0      4
6     3.1      4
7     3.2      2
8     3.3     16
9     3.4     10
10    3.5     26
11    3.6     35
12    3.7     42
13    3.8     86
14    3.9    123
15    4.0    182
16    4.1    244
17    4.2    228
18    4.3    230
19    4.4    123
20    4.5     75
21    4.6     17
22    4.7      6
23    4.8      3
24    5.0      3


In [21]:

c.execute('''
UPDATE amazon_data
SET rating_count = CAST(REPLACE(rating_count, ',', '') AS float);
''')


In [26]:

c.execute('''
    ALTER TABLE amazon_data
    DROP COLUMN img_link;
''')

c.execute('''
    ALTER TABLE amazon_data
    DROP COLUMN product_link;
''')


In [31]:

c.execute('''
    ALTER TABLE amazon_data
    ADD COLUMN category_1 TEXT;
''')

c.execute('''
    ALTER TABLE amazon_data
    ADD COLUMN category_2 TEXT;
''')

c.execute('''
    ALTER TABLE amazon_data
    ADD COLUMN category_3 TEXT;
''')


In [32]:

c.execute('''
    UPDATE amazon_data
SET category_1 = SUBSTR(category, 1, INSTR(category, '|')-1),
   category_2 = CASE WHEN INSTR(category, '|') > 0 THEN SUBSTR(category, INSTR(category, '|')+1)
                      ELSE NULL
                 END,
    category_3 = CASE WHEN INSTR(category_2, '|') > 0 THEN SUBSTR(category_2, INSTR(category_2, '|')+1)
                      ELSE NULL
                 END;
''')



In [39]:

c.execute('''
    ALTER TABLE amazon_data
    ADD COLUMN category TEXT;
''')

In [40]:

c.execute('''
    UPDATE amazon_data
SET category = category_2 
''')

In [41]:

c.execute('''
    UPDATE amazon_data
SET category_2 = SUBSTR(category, 1, INSTR(category, '|')-1)
''')


In [48]:

c.execute('''
    UPDATE amazon_data
SET category_3 = CASE WHEN INSTR(category, '|') > 0 THEN SUBSTR(category, INSTR(category, '|')+1)
                      ELSE NULL
                 END;
''')

In [50]:

c.execute('''
    ALTER TABLE amazon_data
    DROP COLUMN category;
''')

In [35]:

c.execute('''
SELECT category_1, COUNT(*) as count
FROM amazon_data
GROUP BY category_1
ORDER BY count DESC;
''')
category_1_df = pd.DataFrame(c.fetchall(),columns=['Category_1',
'COUNT'])
print(category_1_df)

              Category_1  COUNT
0            Electronics    526
1  Computers&Accessories    453
2           Home&Kitchen    448
3         OfficeProducts     31
4     MusicalInstruments      2
5        HomeImprovement      2
6             Toys&Games      1
7    Health&PersonalCare      1
8          Car&Motorbike      1


In [36]:

c.execute("UPDATE amazon_data SET category_1 = REPLACE(category_1, '&', ' & ') WHERE category_1 LIKE '%&%';")
c.execute("UPDATE amazon_data SET category_1 = 'Office Products' WHERE category_1 = 'OfficeProducts';")
c.execute("UPDATE amazon_data SET category_1 = 'Musical Instruments' WHERE category_1 = 'MusicalInstruments';")
c.execute("UPDATE amazon_data SET category_1 = 'Home Improvement' WHERE category_1 = 'HomeImprovement';")

In [42]:

c.execute('''
SELECT category_2, COUNT(*) as count
FROM amazon_data
GROUP BY category_2
ORDER BY count DESC;
''')
category_2_df = pd.DataFrame(c.fetchall(),columns=['Category_2',
'COUNT'])
print(category_2_df)

                                 Category_2  COUNT
0                   Accessories&Peripherals    381
1                    Kitchen&HomeAppliances    308
2                      HomeTheater,TV&Video    162
3                       Mobiles&Accessories    161
4                Heating,Cooling&AirQuality    116
5                        WearableTechnology     76
6            Headphones,Earbuds&Accessories     66
7                         NetworkingDevices     33
8                       OfficePaperProducts     27
9               ExternalDevices&DataStorage     18
10                 HomeStorage&Organization     16
11                                HomeAudio     16
12                      Cameras&Photography     16
13                              Accessories     14
14                Printers,Inks&Accessories     11
15  GeneralPurposeBatteries&BatteryChargers     10
16                                               8
17                           CraftMaterials      7
18                             

In [49]:

c.execute('''
SELECT category_3, COUNT(*) as count
FROM amazon_data
GROUP BY category_3
ORDER BY count DESC;
''')
category_3_df = pd.DataFrame(c.fetchall(),columns=['Category_3',
'COUNT'])
print(category_3_df)

                                         Category_3  COUNT
0               Cables&Accessories|Cables|USBCables    233
1                                      SmartWatches     76
2              Smartphones&BasicMobiles|Smartphones     68
3                      Televisions|SmartTelevisions     63
4                                 Headphones|In-Ear     52
..                                              ...    ...
203                Accessories|Cables|SpeakerCables      1
204  Accessories|Batteries&Chargers|BatteryChargers      1
205                            Accessories|Adapters      1
206                           Accessories|3DGlasses      1
207                          AVReceivers&Amplifiers      1

[208 rows x 2 columns]


In [56]:


# Replace '&' with ' & '
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, '&', ' & ')
''')

# Replace ',' with ', '
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, ',', ', ')
''')

# Replace 'HomeAppliances' with 'Home Appliances'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'HomeAppliances', 'Home Appliances')
''')

# Replace 'AirQuality' with 'Air Quality'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'AirQuality', 'Air Quality')
''')

# Replace 'WearableTechnology' with 'Wearable Technology'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'WearableTechnology', 'Wearable Technology')
''')

# Replace 'NetworkingDevices' with 'Networking Devices'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'NetworkingDevices', 'Networking Devices')
''')

# Replace 'OfficePaperProducts' with 'Office Paper Products'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'OfficePaperProducts', 'Office Paper Products')
''')

# Replace 'ExternalDevices' with 'External Devices'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'ExternalDevices', 'External Devices')
''')

# Replace 'DataStorage' with 'Data Storage'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'DataStorage', 'Data Storage')
''')

# Replace 'HomeStorage' with 'Home Storage'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'HomeStorage', 'Home Storage')
''')

# Replace 'HomeAudio' with 'Home Audio'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'HomeAudio', 'Home Audio')
''')

# Replace 'GeneralPurposeBatteries' with 'General Purpose Batteries'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'GeneralPurposeBatteries', 'General Purpose Batteries')
''')

# Replace 'BatteryChargers' with 'Battery Chargers'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'BatteryChargers', 'Battery Chargers')
''')

# Replace 'CraftMaterials' with 'Craft Materials'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'CraftMaterials', 'Craft Materials')
''')

# Replace 'OfficeElectronics' with 'Office Electronics'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'OfficeElectronics', 'Office Electronics')
''')

# Replace 'PowerAccessories' with 'Power Accessories'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'PowerAccessories', 'Power Accessories')
''')

# Replace 'CarAccessories' with 'Car Accessories'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2, 'CarAccessories', 'Car Accessories')
''')

# Replace 'HomeMedicalSupplies' with 'Home Medical Supplies'
c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2,'HomeMedicalSupplies','Home Medical Supplies')
''')

c.execute('''
    UPDATE amazon_data
    SET category_2 = REPLACE(category_2,'HomeTheater','Home Theater')
''')


In [57]:

c.execute('''
SELECT category_2, COUNT(*) as count
FROM amazon_data
GROUP BY category_2
ORDER BY count DESC;
''')
category_2_df = pd.DataFrame(c.fetchall(),columns=['Category_2',
'COUNT'])
print(category_2_df)

                                      Category_2  COUNT
0                      Accessories & Peripherals    381
1                      Kitchen & Home Appliances    308
2                       Home Theater, TV & Video    162
3                          Mobiles & Accessories    161
4                 Heating, Cooling & Air Quality    116
5                            Wearable Technology     76
6              Headphones, Earbuds & Accessories     66
7                             Networking Devices     33
8                          Office Paper Products     27
9                External Devices & Data Storage     18
10                   Home Storage & Organization     16
11                                    Home Audio     16
12                         Cameras & Photography     16
13                                   Accessories     14
14                  Printers, Inks & Accessories     11
15  General Purpose Batteries & Battery Chargers     10
16                                              

In [58]:

c.execute('''
UPDATE amazon_data
SET product_id = TRIM(product_id);
''')


In [59]:

c.execute('''
    ALTER TABLE amazon_data
    ADD COLUMN rating_score TEXT;
''')

In [105]:

c.execute('''
UPDATE amazon_data
SET rating_score = 
    CASE 
        WHEN rating < 2.0 THEN 'Poor'
        WHEN rating < 3.0 THEN 'Below Average'
        WHEN rating < 4.0 THEN 'Average'
        WHEN rating < 5.0 THEN 'Above Average'
        ELSE 'Excellent'
    END;
''')


In [63]:

c.execute('''
ALTER TABLE amazon_data ADD COLUMN difference_price FLOAT;
''')

In [65]:

c.execute('''
UPDATE amazon_data
SET difference_price = actual_price - discounted_price;
''')


In [67]:

c.execute('''
    ALTER TABLE amazon_data
    DROP COLUMN category_3;
''')

In [74]:

df = pd.read_sql_query("SELECT * FROM amazon_data", conn)
df.dtypes

product_id              object
product_name            object
discounted_price        object
actual_price            object
discount_percentage     object
rating                  object
rating_count            object
about_product           object
user_id                 object
user_name               object
review_id               object
review_title            object
review_content          object
category_1              object
category_2              object
rating_score            object
difference_price       float64
dtype: object

In [93]:

c.execute('''
PRAGMA table_info(amazon_data);
''')
result = c.fetchall()
for row in result:
    print(row)

(0, 'product_id', 'TEXT', 0, None, 0)
(1, 'product_name', 'TEXT', 0, None, 0)
(2, 'about_product', 'TEXT', 0, None, 0)
(3, 'user_id', 'TEXT', 0, None, 0)
(4, 'user_name', 'TEXT', 0, None, 0)
(5, 'review_id', 'TEXT', 0, None, 0)
(6, 'review_title', 'TEXT', 0, None, 0)
(7, 'review_content', 'TEXT', 0, None, 0)
(8, 'category_1', 'TEXT', 0, None, 0)
(9, 'category_2', 'TEXT', 0, None, 0)
(10, 'discounted_price', 'REAL', 0, None, 0)
(11, 'actual_price', 'REAL', 0, None, 0)
(12, 'discount_percentage', 'REAL', 0, None, 0)
(13, 'rating', 'REAL', 0, None, 0)
(14, 'rating_score', 'REAL', 0, None, 0)
(15, 'difference_price', 'REAL', 0, None, 0)
(16, 'rating_count', 'INTEGER', 0, None, 0)


In [81]:

def convert_columns_to_real(table_name, columns):
    for col in columns:
        c.execute(f"ALTER TABLE {table_name} RENAME COLUMN {col} TO {col}_text;")
        c.execute(f"ALTER TABLE {table_name} ADD COLUMN {col} REAL;")
        c.execute(f"UPDATE {table_name} SET {col} = CAST({col}_text AS REAL);")
        c.execute(f"ALTER TABLE {table_name} DROP COLUMN {col}_text;")


In [82]:

convert_columns_to_real('amazon_data', ['discounted_price','actual_price','discount_percentage','rating','difference_price'])


In [91]:

def convert_columns_to_int(table_name, columns):
    for col in columns:
        c.execute(f"ALTER TABLE {table_name} RENAME COLUMN {col} TO {col}_text;")
        c.execute(f"ALTER TABLE {table_name} ADD COLUMN {col} INTEGER;")
        c.execute(f"UPDATE {table_name} SET {col} = CAST({col}_text AS INTEGER);")
        c.execute(f"ALTER TABLE {table_name} DROP COLUMN {col}_text;")

In [92]:
#code block written by Aazin Shaikh
convert_columns_to_int('amazon_data', ['rating_count'])


## Normalizing cleaned data 

In [108]:

c.execute('''
CREATE TABLE products (
  product_id TEXT PRIMARY KEY,
  product_name TEXT,
  discounted_price REAL,
  actual_price REAL,
  discount_percentage REAL,
  rating REAL,
  rating_count INTEGER,
  about_product TEXT,
  category_1 TEXT,
  category_2 TEXT,
  rating_score TEXT,
  difference_price REAL
);
''')


In [109]:

c.execute('''
INSERT INTO products (
  product_id,
  product_name,
  discounted_price,
  actual_price,
  discount_percentage,
  rating,
  rating_count,
  about_product,
  category_1,
  category_2,
  rating_score,
  difference_price
)
SELECT
  product_id,
  product_name,
  discounted_price,
  actual_price,
  discount_percentage,
  rating,
  rating_count,
  about_product,
  category_1,
  category_2,
  rating_score,
  difference_price
FROM
  amazon_data;
''')




In [101]:

c.execute('''
DELETE FROM amazon_data
WHERE product_id IN (
    SELECT product_id
    FROM amazon_data
    GROUP BY product_id
    HAVING COUNT(*) > 1
);
''')


In [121]:

products_df = pd.read_sql_query("SELECT * FROM products", conn)
products_df

,product_id,product_name,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,category_1,category_2,rating_score,difference_price
0,B008IFXQFU,"TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...",499.0,999.0,0.50,4.2,179691.0,USB WiFi Adapter —— Speedy wireless transmissi...,Computers & Accessories,Networking Devices,Above Average,500.0
1,B0B6F7LX4C,MI 80 cm (32 inches) 5A Series HD Ready Smart ...,13999.0,24999.0,0.44,4.2,32840.0,"Note : The brands, Mi and Xiaomi, are part of ...",Electronics,"Home Theater, TV & Video",Above Average,11000.0
2,B0B1YVCJ2Y,Acer 80 cm (32 inches) I Series HD Ready Andro...,11499.0,19990.0,0.42,4.3,4703.0,Resolution : HD Ready (1366x768) | Refresh Rat...,Electronics,"Home Theater, TV & Video",Above Average,8491.0
3,B08B42LWKN,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,14999.0,19999.0,0.25,4.2,34899.0,Resolution: HD Ready (1366x768) | Refresh Rate...,Electronics,"Home Theater, TV & Video",Above Average,5000.0
4,B0B3MMYHYW,OnePlus 126 cm (50 inches) Y Series 4K Ultra H...,32999.0,45999.0,0.28,4.2,7298.0,Resolution: 4K Ultra HD (3840x2160) | Bezel-le...,Electronics,"Home Theater, TV & Video",Above Average,13000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1254,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,379.0,919.0,0.59,4.0,1090.0,SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,Home & Kitchen,Kitchen & Home Appliances,Above Average,540.0
1255,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,2280.0,3045.0,0.25,4.1,4118.0,"230 Volts, 400 watts, 1 Year",Home & Kitchen,Kitchen & Home Appliances,Above Average,765.0
1256,B009P2LIL4,Bajaj Majesty RX10 2000 Watts Heat Convector R...,2219.0,3080.0,0.28,3.6,468.0,International design and styling|Two heat sett...,Home & Kitchen,"Heating, Cooling & Air Quality",Average,861.0
1257,B00J5DYCCA,Havells Ventil Air DSP 230mm Exhaust Fan (Pist...,1399.0,1890.0,0.26,4.0,8031.0,Fan sweep area: 230 MM ; Noise level: (40 - 45...,Home & Kitchen,"Heating, Cooling & Air Quality",Above Average,491.0


In [130]:

temp_review_df = pd.read_sql_query("SELECT review_id, product_id, user_id, review_title, rating_score FROM amazon_data", conn)

In [131]:

reviewer_id_split = temp_review_df['review_id'].str.split(',', expand=False)

reviewer_id_split

0       [R1LW6NWSVTVZ2H, R3VR5WFKUS15C5, R2F6GC79OYWUK...
1       [R13UTIA6KOF6QV, R2UGDZSGFF01K7, RHHIZ45VYU5X6...
2       [R1EBS3566VCSCG, R24MB66WRPSN2A, R25UU2M1B9BO5...
3       [R3COVVOP2R7Z28, R2T6WHEO2ONNDD, RUFFV2QR43OCM...
4       [R2J3Q3BUHJ2S7E, R2H2ELE1DG24VY, R1U1S7X7BPSZB...
                              ...                        
1254    [R3G3XFHPBFF0E8, R3C0BZCD32EIGW, R2EBVBCN9QPD9...
1255    [R3DDL2UPKQ2CK9, R2SYYU1OATVIU5, R1VM993161IYR...
1256    [R1TLRJVW4STY5I, R2O455KRN493R1, R3Q5MVGBRIAS2...
1257    [R39Q2Y79MM9SWK, R3079BG1NIH6MB, R29A31ZELTZNJ...
1258    [R20RBRZ0WEUJT9, ROKIFK9R2ISSE, R30EEG2FNJSN5I...
Name: review_id, Length: 1259, dtype: object

In [132]:

reviewer_id_exp = reviewer_id_split.explode()

reviewer_id_clean = reviewer_id_exp.reset_index(drop=True)

reviewer_id_clean

0       R1LW6NWSVTVZ2H
1       R3VR5WFKUS15C5
2       R2F6GC79OYWUKQ
3       R3QZ19MECGWG9A
4       R2MPU42MYK7GPO
             ...      
9864    R2ZC03S4QXOW4Y
9865    R186H8YW34BQD5
9866    R10NC3D321N59G
9867     REKF75G4SOAOX
9868    R2G0ZT4JQX322I
Name: review_id, Length: 9869, dtype: object

In [133]:

reviewer_id_clean.to_csv('reviewer_id_clean.csv', index=False)

In [134]:

review_rating_data = pd.read_csv(r'review_rating_data.csv')
review_rating_data.to_sql('review_rating_data', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 'emp'

8444

In [140]:

c.execute('''
CREATE TABLE reviews (
  review_id TEXT PRIMARY KEY,
  product_id TEXT,
  review_content TEXT,
  rating_score TEXT,
  FOREIGN KEY(product_id) REFERENCES products(product_id)
);
''')



In [143]:

c.execute('''
INSERT INTO reviews (
    review_id,
    product_id,
    review_content,
    rating_score
)
SELECT
    review_id,
    product_id,
    review_content,
    rating_score
FROM
  review_rating_data;
''')




In [144]:

temp_rr_df = pd.read_sql_query("SELECT * FROM reviews", conn)
temp_rr_df

,review_id,product_id,review_content,rating_score
0,R1LW6NWSVTVZ2H,B09F3PDDRF,"Compatible with laptop SSD,Good to connect say...",Average
1,R3VR5WFKUS15C5,B07F6GXNPB,"Bigger size,Superb.,Vacuum Bag,Price is reason...",Above Average
2,R2F6GC79OYWUKQ,B0BG62HMDJ,"awesome,Good product,Product reviews ...,Best ...",Above Average
3,R3QZ19MECGWG9A,B07F6GXNPB,"Bigger size,Superb.,Vacuum Bag,Price is reason...",Above Average
4,R2MPU42MYK7GPO,B0BG62HMDJ,"awesome,Good product,Product reviews ...,Best ...",Above Average
...,...,...,...,...
8439,R30EEG2FNJSN5I,B0BG62HMDJ,"awesome,Good product,Product reviews ...,Best ...",Above Average
8440,R2ZC03S4QXOW4Y,B0BG62HMDJ,"awesome,Good product,Product reviews ...,Best ...",Above Average
8441,R186H8YW34BQD5,B08LKS3LSP,"Product is nice,Decent cable,It charges all th...",Average
8442,REKF75G4SOAOX,B07F6GXNPB,"Bigger size,Superb.,Vacuum Bag,Price is reason...",Above Average


## Building Queries

### Query 1: Display the number of products category and subcategory wise

In [146]:

c.execute('''CREATE VIEW product_count_by_category AS
             SELECT category_1 AS main_category, category_2 AS sub_category, COUNT(*) AS product_count
             FROM products
             GROUP BY category_1, category_2''')


In [153]:

c.execute('SELECT main_category, sub_category, product_count FROM product_count_by_category')
product_count_df = pd.DataFrame(c.fetchall(), columns=['Main Category',
'Sub Category',
'Product Count'])
print(product_count_df)

              Main Category                                  Sub Category  \
0           Car & Motorbike                               Car Accessories   
1   Computers & Accessories                                                 
2   Computers & Accessories                     Accessories & Peripherals   
3   Computers & Accessories                                    Components   
4   Computers & Accessories               External Devices & Data Storage   
5   Computers & Accessories                                       Laptops   
6   Computers & Accessories                            Networking Devices   
7   Computers & Accessories                  Printers, Inks & Accessories   
8               Electronics                                                 
9               Electronics                                   Accessories   
10              Electronics                         Cameras & Photography   
11              Electronics  General Purpose Batteries & Battery Chargers   

### Query 2: Display the highest to lowest rated products 

In [157]:
#code block written by Ayesha Mulla
c.execute('''
CREATE VIEW products_rating AS
SELECT product_id as product_id, product_name as product_name, rating AS rating, rating_score AS rating_score, 
FROM products
GROUP BY rating
ORDER BY rating DESC;
''')


In [159]:

c.execute('SELECT product_id,product_name, rating, rating_score FROM products_rating')
products_rating_df = pd.DataFrame(c.fetchall(), columns=['Product ID',
'Product Name',
'Rating',                                                          
'Rating Score'])
print(products_rating_df)

    Product ID                                       Product Name  Rating  \
0   B0BP7XLX48  Syncwire LTG to USB Cable for Fast Charging Co...     5.0   
1   B0BR4F878Q  Swiffer Instant Electric Water Heater Faucet T...     4.8   
2   B09WN3SRC7  Sony Bravia 164 cm (65 inches) 4K Ultra HD Sma...     4.7   
3   B08TGG316Z  10k 8k 4k HDMI Cable, Certified 48Gbps 1ms Ult...     4.6   
4   B00V4BGDKU  TP-Link UE300 USB 3.0 to RJ45 Gigabit Ethernet...     4.5   
5   B014I8SSD0  Amazon Basics High-Speed HDMI Cable, 6 Feet - ...     4.4   
6   B0B1YVCJ2Y  Acer 80 cm (32 inches) I Series HD Ready Andro...     4.3   
7   B008IFXQFU  TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...     4.2   
8   B09ZPM4C2C  TCL 80 cm (32 inches) HD Ready Certified Andro...     4.1   
9   B09NJN8L25  FLiX (Beetel USB to Micro USB PVC Data Sync & ...     4.0   
10  B08Y5KXR6Z  PTron Solero T241 2.4A Type-C Data & Charging ...     3.9   
11  B093ZNQZ2Y  LOHAYA Remote Compatible for Mi Smart LED TV 4...     3.8   

### Query 3: Display the category of products giving highest to lowest discount percentage and corresponding rating 

In [176]:

c.execute('''
CREATE VIEW product_discount AS
SELECT product_id, category_1, category_2, rating_score, discount_percentage
FROM products
GROUP BY discount_percentage
ORDER BY discount_percentage DESC;
''')


In [177]:

c.execute('SELECT product_id, category_1, category_2, rating_score, discount_percentage FROM product_discount')
products_discount_df = pd.DataFrame(c.fetchall(), columns=['Product ID',
'Category',  
'Sub category',                                                      
'Rating Score',
'Discount Percentage'])
print(products_discount_df)

    Product ID                 Category                       Sub category  \
0   B097C564GC  Computers & Accessories          Accessories & Peripherals   
1   B0BF57RN3K              Electronics                Wearable Technology   
2   B09SJ1FTYV              Electronics              Mobiles & Accessories   
3   B0B4T6MR8N  Computers & Accessories          Accessories & Peripherals   
4   B08Y5KXR6Z  Computers & Accessories          Accessories & Peripherals   
..         ...                      ...                                ...   
87  B01F262EUU              Electronics  Headphones, Earbuds & Accessories   
88  B0BD3T6Z1D              Electronics              Mobiles & Accessories   
89  B009UORDX4           Home & Kitchen          Kitchen & Home Appliances   
90  B00LZLQ624          Office Products              Office Paper Products   
91  B071VMP1Z4              Electronics           Home Theater, TV & Video   

     Rating Score  Discount Percentage  
0   Above Average     

### Query 4: Display the category of products having most reviews

In [229]:

c.execute('''
CREATE VIEW product_count_reviews AS
SELECT p.category_1 AS category_1, p.category_2 AS category_2, COUNT(r.review_id) AS review_count
FROM reviews r
INNER JOIN products p on p.product_id = r.product_id
GROUP BY category_1, category_2
ORDER BY review_count DESC;

''')

In [230]:

c.execute('SELECT category_1, category_2, review_count FROM product_count_reviews')
product_count_reviews_df = pd.DataFrame(c.fetchall(), columns=[
'Category',  
'Sub category',                                                                 
'Review Count'])
print(product_count_reviews_df)

                  Category                     Sub category  Review Count
0           Home & Kitchen        Kitchen & Home Appliances          2785
1  Computers & Accessories  External Devices & Data Storage          2556
2  Computers & Accessories        Accessories & Peripherals          2099
3  Computers & Accessories     Printers, Inks & Accessories           605
4           Home & Kitchen   Heating, Cooling & Air Quality           279
5              Electronics         Home Theater, TV & Video           120


### Query 5: Display the count and percentage of reviews recorded for each rating

In [231]:

c.execute('''
CREATE VIEW rating_count_reviews AS
SELECT p.rating AS rating, COUNT(r.review_id) AS review_count, 
(COUNT(r.review_id) * 100.0 / SUM(COUNT(r.review_id)) OVER ()) AS review_percentage
FROM reviews r
INNER JOIN products p on p.product_id = r.product_id
GROUP BY rating
ORDER BY review_count DESC;

''')

In [233]:

c.execute('SELECT rating, review_count, review_percentage FROM rating_count_reviews')
rating_count_reviews_df = pd.DataFrame(c.fetchall(), columns=[
'Rating',  
'Review Count',                                                                 
'Review Percentage'])
print(rating_count_reviews_df)

   Rating  Review Count  Review Percentage
0     4.3          5114          60.563714
1     3.9          1928          22.832781
2     4.0           683           8.088584
3     4.1           279           3.304121
4     3.7           211           2.498816
5     4.2           120           1.421127
6     4.5           109           1.290857


### Query 6: Display the categorywise discount percentage offered and reviews written for corresponding category

In [258]:

c.execute('''
CREATE VIEW category_discount_reviews AS
SELECT p.category_1 AS category_1, p.category_2 AS category_2, p.discount_percentage AS discount_percentage, p.rating_score AS rating_score, r.review_content AS review_content
FROM products p
INNER JOIN reviews r ON p.product_id = r.product_id
ORDER BY discount_percentage DESC;
''')


In [259]:

c.execute('SELECT category_1, category_2, discount_percentage,rating_score,review_content FROM category_discount_reviews')
category_discount_reviews_df = pd.DataFrame(c.fetchall(), columns=[
'Main Category',  
'Sub category',                                                                 
'Discount Percentage',
'Rating Score',
'Review Content'])
print(category_discount_reviews_df)

                Main Category                  Sub category  \
0     Computers & Accessories     Accessories & Peripherals   
1     Computers & Accessories     Accessories & Peripherals   
2     Computers & Accessories     Accessories & Peripherals   
3     Computers & Accessories     Accessories & Peripherals   
4     Computers & Accessories     Accessories & Peripherals   
...                       ...                           ...   
8439  Computers & Accessories  Printers, Inks & Accessories   
8440  Computers & Accessories  Printers, Inks & Accessories   
8441  Computers & Accessories  Printers, Inks & Accessories   
8442  Computers & Accessories  Printers, Inks & Accessories   
8443  Computers & Accessories  Printers, Inks & Accessories   

      Discount Percentage Rating Score  \
0                    0.65      Average   
1                    0.65      Average   
2                    0.65      Average   
3                    0.65      Average   
4                    0.65      Av

In [ ]:

conn.close()

## References:
### 1.	https://learn.microsoft.com/en-us/sql/relational-databases/tables/primary-and-foreign-key-constraints?view=sql-server-ver16 - for primary key and foreign key constraints and general understanding of concepts
### 2.	https://www.sqlshack.com/ - for relationships definitions
### 3.	https://www.w3schools.com/sql/ - for syntax of all the sql queries
